Load video metadata from a CSV file.

In [ ]:
import pandas as pd
# Load data
video_data = pd.read_csv('video_data.csv')

Extract frames from the video using OpenCV.

In [ ]:
import cv2
# Extract frames from video
frames = []
cap = cv2.VideoCapture('video.mp4')
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    frames.append(frame)
cap.release()

Convert frames to grayscale for consistency.

In [ ]:
# Data preprocessing (e.g., conversion to grayscale)
preprocessed_frames = [cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) for frame in frames]

Resize frames to 224x224 pixels for model input.

In [ ]:
import cv2
# Resize images
resized_frames = [cv2.resize(frame, (224, 224)) for frame in preprocessed_frames]

Convert labels to a one-hot encoded format.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# One-Hot Encoding
encoder = OneHotEncoder(sparse=False)
categorical_labels = encoder.fit_transform(video_data['label'].values.reshape(-1, 1))

Split the dataset into training and testing sets.

In [ ]:
from sklearn.model_selection import train_test_split
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(resized_frames, categorical_labels, test_size=0.2)

Apply data augmentation techniques to training data.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# Data Augmentation
generator = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
augmented_data = generator.flow(X_train, y_train)

Define a simple Convolutional Neural Network for classification.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
# Create Video Classification Model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(categorical_labels.shape[1], activation='softmax'))

Compile the model with the Adam optimizer and categorical crossentropy loss.

In [ ]:
# Compile Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Train the model on the augmented dataset.

In [ ]:
# Train Model
history = model.fit(augmented_data, epochs=10, validation_data=(X_test, y_test))

Evaluate the model's performance on the test set.

In [ ]:
# Evaluate Model
loss, accuracy = model.evaluate(X_test, y_test)

Visualize training and validation accuracy.

In [ ]:
import matplotlib.pyplot as plt
# Plot Training Results
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

Save the trained model to a file.

In [ ]:
from keras.models import save_model
# Serialize Model
save_model(model, 'video_classification_model.h5')

Predict whether the video contains violence using the model.

In [ ]:
# Predict Video for Violence
predictions = model.predict(resized_frames)
violence_predictions = predictions.argmax(axis=1)